In [1]:
import pandas as pd

In [2]:
M = 10 ** 6
E = 1 / M

In [3]:
# lambda x: 1 if x >= 1 else 0

In [4]:
def test(r1, r2):
    x_list = []
    f_list = []
    r1_list = []
    r2_list = []
    r1_and_r2_list = []
    for x in [0, 0.5, 1, 1.5]:
        for f in [0, 1]:
            r1_res = r1(x, f)
            r2_res = r2(x, f)
            r1_and_r2_res = r1_res and r2_res
            x_list.append(x)
            f_list.append(f)
            r1_list.append(r1_res)
            r2_list.append(r2_res)
            r1_and_r2_list.append(r1_and_r2_res)
            
    dictionary = {
        "x": x_list,
        "f": f_list,
        "r1": r1_list,
        "r2": r2_list,
        "r1 && r2": r1_and_r2_list
    }
    df = pd.DataFrame(dictionary)
    return df

In [5]:
r1 = lambda x, f: x - f >= 0

In [6]:
r2 = lambda x, f: x + E - M * f <= 1

In [7]:
test(r1, r2)

,x,f,r1,r2,r1 && r2
0,0.0,0,True,True,True
1,0.0,1,False,True,False
2,0.5,0,True,True,True
3,0.5,1,False,True,False
4,1.0,0,True,False,False
5,1.0,1,True,True,True
6,1.5,0,True,False,False
7,1.5,1,True,True,True
